# pinn class

In [1]:
import torch
import torch.nn as nn
import numpy as np


class PINN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(PINN, self).__init__()
        self.layers = nn.ModuleList(
            [
                (
                    nn.Linear(input_size if i == 0 else hidden_size, hidden_size)
                    if i % 2 == 0
                    else nn.Tanh()
                )
                for i in range(20)
            ]
        )
        self.layers.append(nn.Linear(hidden_size, output_size))
        self.loss = nn.MSELoss()
        self.lambda2 = nn.Parameter(
            torch.tensor([0.01], dtype=torch.float32, device="cuda")
        )
        self.optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        self.optimizer.param_groups[0]["params"].append(self.lambda2)

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

    def loss_fn(self, x, u):
        u_pred = self.forward(x)
        return self.loss(u_pred, u)

    def residual_loss(self, xtrain, fhat):
        x = xtrain[:, 0]
        t = xtrain[:, 1]
        g = xtrain.clone()
        g.requires_grad = True
        u_pred = self.forward(g)
        u_x_t = torch.autograd.grad(
            u_pred,
            g,
            torch.ones([xtrain.shape[0], 1]).to("cuda"),
            retain_graph=True,
            create_graph=True,
        )[0]
        u_xx_tt = torch.autograd.grad(
            u_x_t, g, torch.ones(xtrain.shape).to("cuda"), create_graph=True
        )[0]
        u_x = u_x_t[:, [0]]
        u_t = u_x_t[:, [1]]
        u_xx = u_xx_tt[:, [0]]
        return self.loss(u_t + (u_pred * u_x) - (self.lambda2 * u_xx), fhat)

    def total_loss(self, xtrain, utrain, fhat):
        return self.loss_fn(xtrain, utrain) + self.residual_loss(xtrain, fhat)

    def train_model(self, xtrain, utrain, epochs=1000):
        fhat = torch.zeros(xtrain.shape[0], 1, device="cuda")
        for epoch in range(epochs):
            self.optimizer.zero_grad()
            loss = self.total_loss(xtrain, utrain, fhat)
            loss.backward()
            self.optimizer.step()
            if epoch % 1000 == 0:
                print(f"Epoch {epoch}, Loss {loss.item()}, Lambda2 (Nu) {self.lambda2.item()}")
        return loss.item()


model = PINN(input_size=2, hidden_size=20, output_size=1).to("cuda")
print(model)

PINN(
  (layers): ModuleList(
    (0): Linear(in_features=2, out_features=20, bias=True)
    (1): Tanh()
    (2): Linear(in_features=20, out_features=20, bias=True)
    (3): Tanh()
    (4): Linear(in_features=20, out_features=20, bias=True)
    (5): Tanh()
    (6): Linear(in_features=20, out_features=20, bias=True)
    (7): Tanh()
    (8): Linear(in_features=20, out_features=20, bias=True)
    (9): Tanh()
    (10): Linear(in_features=20, out_features=20, bias=True)
    (11): Tanh()
    (12): Linear(in_features=20, out_features=20, bias=True)
    (13): Tanh()
    (14): Linear(in_features=20, out_features=20, bias=True)
    (15): Tanh()
    (16): Linear(in_features=20, out_features=20, bias=True)
    (17): Tanh()
    (18): Linear(in_features=20, out_features=20, bias=True)
    (19): Tanh()
    (20): Linear(in_features=20, out_features=1, bias=True)
  )
  (loss): MSELoss()
)


# load data

In [2]:
import torch
import numpy as np
def loadAndPrep(u):
    x = [i for i in range(201)]
    t = [i for i in range(1024)]
    x = torch.tensor(x, dtype=torch.float32)
    t = torch.tensor(t, dtype=torch.float32)
    u = torch.tensor(u, dtype=torch.float32)
    X, T = np.meshgrid(x, t)
    xtrue = np.hstack((X.flatten()[:, None], T.flatten()[:, None]))
    idx = np.random.choice(201*1024, 10000, replace=False)
    xtrain = xtrue[idx, :]
    utrain = u.flatten()[idx][:, None]
    device = torch.device("cuda")
    xtrain = torch.tensor(xtrain, dtype=torch.float32, device=device)
    xtrue = torch.tensor(xtrue, dtype=torch.float32, device=device)
    utrain = utrain.to(device)
    utrue = u.flatten()[:, None]
    return xtrain, xtrue, utrain, utrue

In [3]:
import gc
def trainAndLog(u):
    xtrain, _, utrain, _ = loadAndPrep(u)
    model = PINN(input_size=2, hidden_size=20, output_size=1).to("cuda")
    loss = model.train_model(xtrain, utrain, epochs=15000)
    l = model.lambda2.item()
    del model
    gc.collect()
    torch.cuda.empty_cache()
    return l, loss

In [5]:
import h5py
import random
import json

random.seed(69)
d = {}
with h5py.File("/kaggle/input/burgers-clean/simulation_data.h5", "r") as f:
    a = random.choices(list(f.keys()), k=50)
    n = 0
    for i in a:
        print(n)
        d[i] = []
        d[i].append({"epsilon" : f[i]["epsilon"][()]})
        uclean = f[i]["clean"][:]
        pred, loss = trainAndLog(uclean)
        d[i].append({"clean" : [{"predicted" : pred, "loss" : loss}]})
        unoisy = f[i]["noisy"][:]
        pred, loss = trainAndLog(unoisy)
        d[i].append({"noisy": [{"predicted" : pred, "loss" : loss}]})
        n += 1
        with open("results.json", "w") as g:
            json.dump(d, g)

0
Epoch 0, Loss 0.22437208890914917, Lambda2 (Nu) 0.010001188144087791
Epoch 0, Loss 0.3579258322715759, Lambda2 (Nu) 0.009996850974857807
1
Epoch 0, Loss 0.32747524976730347, Lambda2 (Nu) 0.009999974630773067
Epoch 0, Loss 0.3455558717250824, Lambda2 (Nu) 0.009993717074394226
2
Epoch 0, Loss 0.3950692415237427, Lambda2 (Nu) 0.010007492266595364


KeyboardInterrupt: 